# Clustering Notebook

The following notebook can be used in order to generate population settlements. The final result will include this 6 columns.

**1.** id – The IDs are given as a unique number for each cluster. This enables the user to process the data contained in the clusters outside of a GIS software and then merge the data with the clusters.

**2.** Country - Name of the country.

**3.** Population – This is the population in each cluster obtained from the population dataset.

**4.** NightLight – This value is obtained from the night-time light map and represents the maximum luminance detected in each cluster based on the latest stable light product available.

**5.** ElecPop – The number of people in each cluster who live in areas in which the stable light product detect light sources.

**6.** Area – The area of each cluster given in sq. kilometres.
    


## Cell 1 - Importing packages Datasets

In [1]:
from ipynb.fs.full.funcs3 import *

## Cell 1 - Selecting Datasets

Select the workspace, this is the folder that will be used for the outputs. 

**NOTE** Select an empty folder as all the files will be deleted from the workspace once the clusters are generated

You will also have to select the three datasets used in the analysis. These are:

**Administrative boundaries**
This should be disagreggated. It will be used to 1) delimit the population layer to the area of interest and 2) to limit the maximum size of the clusters

**Population raster**

**Night time lights**
This will be used in order to determine how many people live in areas with visible lights. 


In [2]:
messagebox.showinfo('OnSSET extraction', 'Output folder')
workspace = filedialog.askdirectory()

messagebox.showinfo('OnSSET', 'Select the population map')
filename_pop = filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*")))
pop=gdal.Open(filename_pop)
poprasterio=rasterio.open(filename_pop)

messagebox.showinfo('OnSSET', 'Select the nightlights map')
filename_NTL = filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*")))
NTL = gdal.Open(filename_NTL)
NTLrasterio = rasterio.open(filename_NTL)

messagebox.showinfo('OnSSET', 'Select the admin map')
filename_admin = (filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))
admin=gpd.read_file(filename_admin)

## Cell 2 - Setting study area name

This will dictate the name in the final clusters

In [3]:
country_name = "Yemen"

## Cell 3 - Setting the target coordinate system
When calculating distances it is important to choose a coordinate system that represents distances correctly in your area of interst. The coordinate system that is given below is the World Mercator, these coordinate system works well for Sub Saharan Africa but the distortions get larger as you move away from the equator.

In order to select your own coordinate system go to [epsg.io](http://epsg.io/) and type in your area of interest, this will give you a list of coordinate systems to choose from. Once you have selected your coordinate system replace the numbers below with the numbers from your coordinate system **(keep the "EPSG" part)**.

**NOTE** When selecting your coordinate system make sure that you select a system with the unit of meters, this is indicated for all systems on [epsg.io](http://epsg.io/)

In [4]:
crs = 'EPSG:3395'

## Cell 4 - Clipping nighttime lights

In [5]:
clipped_NTL = clipRaster(NTLrasterio.name, filename_admin)

## Cell 5 - Reclassifying rasters

In [6]:
reclassified_NTL = reclassifyRasters(clipped_NTL) 
reclassified_pop = reclassifyRasters(pop) 

clipped_pop = resampleRaster(reclassified_pop, 0.00083333, 0.00083333)
clipped_NTL = resampleRaster(reclassified_NTL, NTLrasterio.transform[0], NTLrasterio.transform[4])

## Cell 6 - Convert to polygon and generate the ElecPop raster

In [ ]:
toPolygon(clipped_pop, 1, workspace)
toPolygon(clipped_NTL, 2, workspace)

elecPop = clip_and_save_raster(poprasterio.name, workspace + r"/NTLArea.shp", "EPSG:4326",workspace)

## Cell 7 - Buffering the clusters

In [ ]:
buffered = buffer(workspace + r"\clusters.shp", workspace) 

## Cell - 8 Covert clusters to singlepart polygons

In [ ]:
single = multi2single(buffered, workspace)

## Cell 9 - Dissolving and splitting the clusters

In [ ]:
clusters = dissolveandsplit(single, crs,filename_admin, country_name)

## Cell 10 - Populating clusters

In [ ]:
clusters = populatingClusters(clusters, poprasterio, "Pop", "sum")
clusters = populatingClusters(clusters, NTLrasterio, "NTL", "max")
clusters = populatingClusters(clusters, elecPop, "ElecPop", "sum")

## Cell 11 - Finalizing clusters

In [ ]:
elecPop = None
finalizing_clusters(clusters, workspace)